In [ ]:
# Code to annotate variants with VEP on UKB RAP
# initiate hail and spark
from pyspark.sql import SparkSession
import hail as hl
builder = (
    SparkSession
    .builder
    .enableHiveSupport()
)
spark = builder.getOrCreate()
hl.init(sc=spark.sparkContext)
import dxpy

# build database on RAP
db_name = "ukb23157_pvcf_annotation"
stmt = f"CREATE DATABASE IF NOT EXISTS {db_name} LOCATION 'dnax://'" 
spark.sql(stmt).show()

# vep annotation
for i in block_list:
    # read pvcf file
    file_url =  "file:///mnt/project/Bulk/Exome sequences/Population level exome OQFE variants, pVCF format - final release/"+str(i)+"_v1.vcf.gz"
    mt = hl.import_vcf(file_url, force_bgz=True, reference_genome="GRCh38", array_elements_required=False)
    # annotate
    annotated_mt = hl.vep(mt,"file:///opt/notebooks/vep-GRCh38.json", csq=True)
    # upload annotated mt to database
    mt_name = str(i)+"_annotated_mt"
    db_uri = dxpy.find_one_data_object(name=f"{db_name}",classname="database")['id']
    url = f"dnax://{db_uri}/{mt_name}"
    annotated_mt.write(url,overwrite=True)